## Backtracking Template

In [1]:
def backtrack(candidate):
    if find_solution(candidate):
        output(candidate)
        return
    
    # iterate all possible candidates.
    for next_candidate in list_of_candidates:
        if is_valid(next_candidate):
            # try this partial candidate solution
            place(next_candidate)
            # given the candidate, explore further.
            backtrack(next_candidate)
            # backtrack
            remove(next_candidate)

## 79. Word Search

### self-written solution after checking answers

In [3]:
from typing import List

In [10]:
class Solution79:
    def exist(self, board: List[List[str]], word: str) -> bool:
        nrow, ncol = len(board), len(board[0])
        def backtrack(row, col, i):
#             print(row, col, i)
            if i == len(word):
                return True
            if row == -1 or row == nrow or col == -1 or col == ncol or board[row][col] != word[i]:
                return False
            # this indicates that board[row][col] == word[i]
            ret = False
            board[row][col] = ''
            for nextrow, nextcol in [(row, col+1), (row, col-1), (row+1, col), (row-1, col)]:
                if backtrack(nextrow, nextcol, i+1):
                    ret = True
                    break
            board[row][col] = word[i]
            return ret
        for i in range(nrow):
            for j in range(ncol):
                if backtrack(i, j, 0):
                    return True
        return False

In [11]:
solver_79 = Solution79()
solver_79.exist([["A","B","C","E"],["S","F","C","S"],["A","D","E","E"]], "ABCCED")

True

## 51. N-Queens

### self-written solution after checking answers

In [12]:
class Solution51:
    def solveNQueens(self, n: int) -> List[List[str]]:
        def addQueens(row, col):
            columns[col] = 1
            diagonals[row-col] = 1
            antidiagonals[row+col] = 1
            queens.append(col)
        def removeQueens(row, col):
            columns[col] = 0
            diagonals[row-col] = 0
            antidiagonals[row+col] = 0
            queens.pop()
        def addSolution():
            ret.append(['.' * col + 'Q' + '.' * (n- col -1) for col in queens])
        def backtrack(row=0):
            for col in range(n):
                if not columns[col] and not diagonals[row-col] and not antidiagonals[row+col]:
                    addQueens(row, col)
                    if row == n - 1:
                        addSolution()
                    else:
                        backtrack(row+1)
                    removeQueens(row, col)
        columns = [0] * n
        diagonals = [0] * (2*n-1)
        antidiagonals = [0] * (2*n-1)
        queens = []
        ret = []
        backtrack()
        return ret

In [18]:
solver_51 = Solution51()
len(solver_51.solveNQueens(8))

92

## 22. Generate Parentheses

### self-written recursive solution after checking answers

In [20]:
class Solution22:
    def generateParenthesis(self, n: int) -> List[str]:
        ret = []
        def backtrack(curr='', left=0, right=0):
            if len(curr) == 2*n:
                ret.append(curr)
                return
            if left < n:
                backtrack(curr+'(', left+1, right)
            if right < left:
                backtrack(curr+')', left, right+1)
        backtrack()
        return ret

### self-written bottom-up dynamic programming after checking answers

In [21]:
class Solution22:
    def generateParenthesis(self, n: int) -> List[str]:
        dp = [[''] for _ in range(n+1)]
        for i in range(1, n+1):
            dp[i] = ['('+s1+')'+s2 for j in range(i) for s1 in dp[j] for s2 in dp[i-1-j]]
        return dp[n]

## 211. Design Add and Search Words Data Structure

### self-written recursive solution, passed all tests

In [25]:
class WordDictionary:

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.root = {}

    def addWord(self, word: str) -> None:
        node = self.root
        for char in word:
            if char not in node:
                node[char] = {}
            node = node.get(char)
        node['#'] = True # True is not important here actually

    def search(self, word: str) -> bool:
        def helper(node, i):
            if i == len(word):
                return '#' in node
            if word[i] == '.':
                for char, child_node in node.items():
                    if char == '#':
                        continue
                    if helper(child_node, i+1):
                        return True
                return False
            if word[i] in node:
                return helper(node.get(word[i]), i+1)
            return False
        return helper(self.root, 0)

In [26]:
wd = WordDictionary()
wd.addWord("bad")
wd.addWord("dad")
wd.addWord("mad")
wd.search("pad"), wd.search("bad"), wd.search(".ad"), wd.search("b..")

(False, True, True, True)

## 212. Word Search II

### self-written solution after checking answers, passed all tests

In [40]:
class Solution212:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # build trie first
        trie = {}
        for word in words:
            node = trie
            for char in word:
                if char not in node:
                    node[char] = {}
                node = node.get(char)
            node['#'] = word # change True to word to facilitate collection
#         print(trie)
        ret = []
        nrow, ncol = len(board), len(board[0])
        
        def backtrack(row, col, node):
            # end condition
            if '#' in node:
                # get and delete in one step
                ret.append(node.pop('#'))
#                 ret.append(node.get('#'))
                # delete to prevent multiple same outputs
#                 del node['#']
                # can't put return here! dogs and dogs
            if row == -1 or row == nrow or col == -1 or col == ncol or not board[row][col]:
                return
            char = board[row][col]
            if char in node:
                childnode = node.get(char)
#                 print(childnode)
                board[row][col] = ''
                for nextrow, nextcol in [(row+1, col), (row-1, col), (row, col+1), (row, col-1)]:
                    backtrack(nextrow, nextcol, childnode)
                board[row][col] = char
            # otherwise do nothing
        
        for i in range(nrow):
            for j in range(ncol):
                backtrack(i, j, trie)
        
        return ret

In [41]:
solver_212 = Solution212()
solver_212.findWords([["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]], 
                     ["oath","pea","eat","rain"])

['oath', 'eat']

### self-written improved solution: optimization with pruning, very fast

In [50]:
class Solution212:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # build trie first
        trie = {}
        for word in words:
            node = trie
            for char in word:
                if char not in node:
                    node[char] = {}
                node = node.get(char)
            node['#'] = word
#         print(trie)
        
        ret = []
        nrow, ncol = len(board), len(board[0])
        
        def backtrack(row, col, parent):
            # it is guaranteed that such child node exists
            char = board[row][col]
            nextNode = parent[char]
            if '#' in nextNode:
                ret.append(nextNode.pop('#'))
            board[row][col] = ''
            for nextrow, nextcol in [(row+1, col), (row-1, col), (row, col+1), (row, col-1)]:
                if nextrow != -1 and nextrow != nrow and nextcol != -1 and nextcol != ncol and board[nextrow][nextcol] in nextNode:
                    backtrack(nextrow, nextcol, nextNode)
            board[row][col] = char
            # pruning operation
            if not nextNode:
                parent.pop(char)
        
        for row in range(nrow):
            for col in range(ncol):
                # starting from each of the cells
                if board[row][col] in trie:
                    backtrack(row, col, trie)
        
#         print(trie)
        
        return ret

In [51]:
solver_212 = Solution212()
solver_212.findWords([["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]], 
                     ["oath","pea","eat","rain"])

{'o': {'a': {'t': {'h': {'#': 'oath'}}}}, 'p': {'e': {'a': {'#': 'pea'}}}, 'e': {'a': {'t': {'#': 'eat'}}}, 'r': {'a': {'i': {'n': {'#': 'rain'}}}}}
{'p': {'e': {'a': {'#': 'pea'}}}, 'r': {'a': {'i': {'n': {'#': 'rain'}}}}}


['oath', 'eat']

## 46. Permutations

### self-written backtracking solution after checking answers

In [55]:
class Solution46:
    def permute(self, nums: List[int]) -> List[List[int]]:
        ret = []
        def backtrack(first, nums):
            if first == len(nums):
                ret.append(nums)
            for i in range(first, len(nums)):
                new_nums = nums[:]
                new_nums[i], new_nums[first] = new_nums[first], new_nums[i]
                backtrack(first+1, new_nums)
        backtrack(0, nums)
        return ret

In [57]:
solver_46 = Solution46()
solver_46.permute([1, 2, 3]), solver_46.permute([5, 7, 8, 1])

([[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 2, 1], [3, 1, 2]],
 [[5, 7, 8, 1],
  [5, 7, 1, 8],
  [5, 8, 7, 1],
  [5, 8, 1, 7],
  [5, 1, 8, 7],
  [5, 1, 7, 8],
  [7, 5, 8, 1],
  [7, 5, 1, 8],
  [7, 8, 5, 1],
  [7, 8, 1, 5],
  [7, 1, 8, 5],
  [7, 1, 5, 8],
  [8, 7, 5, 1],
  [8, 7, 1, 5],
  [8, 5, 7, 1],
  [8, 5, 1, 7],
  [8, 1, 5, 7],
  [8, 1, 7, 5],
  [1, 7, 8, 5],
  [1, 7, 5, 8],
  [1, 8, 7, 5],
  [1, 8, 5, 7],
  [1, 5, 8, 7],
  [1, 5, 7, 8]])

### official solution

In [58]:
class Solution46:
    def permute(self, nums: List[int]) -> List[List[int]]:
        def backtrack(first = 0):
            # if all integers are used up
            if first == n:  
                output.append(nums[:])
            for i in range(first, n):
                # place i-th integer first 
                # in the current permutation
                # this is the placement step
                nums[first], nums[i] = nums[i], nums[first]
                # use next integers to complete the permutations
                backtrack(first + 1)
                # backtrack
                # this is the remove step
                nums[first], nums[i] = nums[i], nums[first]
        
        n = len(nums)
        output = []
        backtrack()
        return output

## 526. Beautiful Arrangement

### self-written backtracking solution, passed all tests

In [106]:
class Solution526:
    def countArrangement(self, n: int) -> int:
        """
        it is important to check only the validity at position 'first'
        position at i may be changed later on (haven't fixed yet)
        """
        nums = [i for i in range(1, n+1)]
        count = 0
#         ret = []
        def check(num1, num2):
            return num1 % num2 == 0 or num2 % num1 == 0
        def backtrack(first=0):
            if first == n:
#                 ret.append(tuple(nums))
                nonlocal count
                count += 1
#                 print(nums)
                return
            for i in range(first, n):
#                 print('first:', first, 'i:', i)
#                 print(nums[i], first+1, nums[first], i+1)
#                 if check(nums[i], first+1) and check(nums[first], i+1):
                if check(nums[i], first+1):
                    nums[first], nums[i] = nums[i], nums[first]
                    backtrack(first + 1)
                    nums[first], nums[i] = nums[i], nums[first]
        backtrack()
#         return ret
        return count
    def check(self, num1, num2):
        return num1 % num2 == 0 or num2 % num1 == 0
    def filterValid(self, perms, n):
        ret = []
        for perm in perms:
            valid = True
            for i in range(n):
                if not self.check(i+1, perm[i]):
                    valid = False
                    break
            if valid:
                ret.append(tuple(perm))
        return ret

In [107]:
solver_526 = Solution526()
solver_526.countArrangement(2), solver_526.countArrangement(5), solver_526.countArrangement(6)

(2, 10, 36)

#### for debugging

In [103]:
set(solver_526.filterValid(solver_46.permute([1, 2, 3, 4, 5, 6]), 6)) - set(solver_526.countArrangement(6))

{(3, 1, 6, 4, 5, 2), (4, 6, 1, 2, 5, 3)}

## 39. Combination Sum (Review)

### self-written backtracking solution

In [108]:
class Solution39:
    def combinationSum(self, candidates: List[int], target: int) -> List[List[int]]:
        ret = []
        def backtrack(choices, index, curr):
            if curr == target:
                ret.append(choices)
            if index == len(candidates):
                return
            if curr < target:
                # add the current
                backtrack(choices + [candidates[index]], index, curr + candidates[index])
                # skip the current
                backtrack(choices, index + 1, curr)
        backtrack([], 0, 0)
        return ret

In [110]:
solver_39 = Solution39()
solver_39.combinationSum(candidates = [2,3,6,7], target = 7),\
solver_39.combinationSum(candidates = [2,3,5], target = 8),\
solver_39.combinationSum(candidates = [2], target = 1),\
solver_39.combinationSum(candidates = [1], target = 2)

([[2, 2, 3], [7]], [[2, 2, 2, 2], [2, 3, 3], [3, 5]], [], [[1, 1]])

### self-written backtracking solution with placement and removal, faster

In [111]:
class Solution39:
    def combinationSum(self, candidates: List[int], target: int) -> List[List[int]]:
        ret = []
        def backtrack(choices, index, curr):
            if curr == target:
                ret.append(choices[:])
            if curr > target:
                return
            for i in range(index, len(candidates)):
                # placement
                choices.append(candidates[i])
                backtrack(choices, i, curr + candidates[i])
                # removal
                choices.pop()
        backtrack([], 0, 0)
        return ret

In [112]:
solver_39 = Solution39()
solver_39.combinationSum(candidates = [2,3,6,7], target = 7),\
solver_39.combinationSum(candidates = [2,3,5], target = 8),\
solver_39.combinationSum(candidates = [2], target = 1),\
solver_39.combinationSum(candidates = [1], target = 2)

([[2, 2, 3], [7]], [[2, 2, 2, 2], [2, 3, 3], [3, 5]], [], [[1, 1]])

## 78. Subsets

### self-written backtracking solution

In [123]:
class Solution78:
    def subsets(self, nums: List[int]) -> List[List[int]]:
        ret = []
        choices = []
        def backtrack(index=0):
            ret.append(choices[:])
            for i in range(index, len(nums)):
                choices.append(nums[i])
                backtrack(i + 1)
                choices.pop()
        backtrack()
        return ret

In [124]:
solver_78 = Solution78()
solver_78.subsets([1,2,3])

[[], [1], [1, 2], [1, 2, 3], [1, 3], [2], [2, 3], [3]]

### self-written top-down dynamic programming solution, damn fast

In [127]:
class Solution78:
    def subsets(self, nums: List[int]) -> List[List[int]]:
        ret = []
        def helper(index):
            if index == -1:
                return [[]]
            subsets = helper(index-1)
            length = len(subsets)
            for i in range(length):
                subsets.append(subsets[i] + [nums[index]])
            return subsets
        return helper(len(nums) - 1)

In [128]:
solver_78 = Solution78()
solver_78.subsets([1,2,3])

[[], [1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3]]

### self-written bottom-up dynamic programing solution

In [138]:
class Solution78:
    def subsets(self, nums: List[int]) -> List[List[int]]:
        ret = [[]]
        for num in nums:
            length = len(ret)
            for i in range(length):
                ret.append(ret[i] + [num])
        return ret

In [139]:
solver_78 = Solution78()
solver_78.subsets([1,2,3])

[[], [1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3]]

## 90. Subsets II

### self-written bottom-up dynamic programming solution

In [148]:
class Solution90:
    def subsetsWithDup(self, nums: List[int]) -> List[List[int]]:
        """
        misunderstood the problem.....
        """
        visited = set()
        ret = [[]]
        for num in nums:
            if num not in visited:
                visited.add(num)
                length = len(ret)
                for i in range(length):
                    ret.append(ret[i] + [num])
        return ret

In [149]:
solver_90 = Solution90()
solver_90.subsetsWithDup([1,2,3,1,2,3,3,2,1])

[[], [1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3]]

### problems of previous approach (duplicates)

In [150]:
class Solution90:
    def subsetsWithDup(self, nums: List[int]) -> List[List[int]]:
        ret = [[]]
        for num in nums:
            length = len(ret)
            for i in range(length):
                ret.append(ret[i] + [num])
        return ret

In [151]:
solver_90 = Solution90()
solver_90.subsetsWithDup([1,2,2])

[[], [1], [2], [1, 2], [2], [1, 2], [2, 2], [1, 2, 2]]

### backtrack solution in discussion area

In [155]:
class Solution90:
    def subsetsWithDup(self, nums: List[int]) -> List[List[int]]:
        def backtrack(start, subset):
            ret.append(subset)
            for i in range(start, len(nums)):
                if i > 0 and nums[i] == nums[i-1]:
                    continue
                backtrack(start + 1, subset + [nums[i]])
        ret = []
        nums.sort()
        backtrack(0, [])
        return ret

In [156]:
solver_90 = Solution90()
solver_90.subsetsWithDup([1, 2, 2, 2])

[[], [1], [1, 2], [2], [2, 2]]

In [157]:
class Solution90:
    def subsetsWithDup(self, nums: List[int]) -> List[List[int]]:
        """
        interesting solution, details remain to be understood
        """
        def backtrack(nums, path):
            res.append(path)
            for i in range(len(nums)):
                if i > 0 and nums[i] == nums[i-1]:
                    continue
                backtrack(nums[i+1:], path + [nums[i]])
        nums.sort()
        res = []
        backtrack(nums, [])
        return res

In [158]:
solver_90 = Solution90()
solver_90.subsetsWithDup([1, 2, 2, 2])

[[], [1], [1, 2], [1, 2, 2], [1, 2, 2, 2], [2], [2, 2], [2, 2, 2]]

## 37. Sudoku Solver

### self-written backtrack solution with the help of sub-box tip, passed all tests, genius!!!

In [169]:
class Solution37:
    def solveSudoku(self, board: List[List[str]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        def getSubBox(row, col):
            return (row // 3) * 3 + col // 3
        def addNumber(row, col, number):
            board[row][col] = str(number)
            rows[row][number-1] = 1
            cols[col][number-1] = 1
            sub_boxes[getSubBox(row, col)][number-1] = 1
        def removeNumber(row, col, number):
            board[row][col] = '.'
            rows[row][number-1] = 0
            cols[col][number-1] = 0
            sub_boxes[getSubBox(row, col)][number-1] = 0
        def checkNumber(row, col, number):
            return not rows[row][number-1] and \
                   not cols[col][number-1] and \
                   not sub_boxes[getSubBox(row, col)][number-1]
        def getNextCell(row, col):
            if col == 8:
                return row + 1, 0
            else:
                return row, col + 1
        # constraints
        rows = [[0 for _ in range(9)] for _ in range(9)]
        cols = [[0 for _ in range(9)] for _ in range(9)]
        sub_boxes = [[0 for _ in range(9)] for _ in range(9)]
        for row in range(9):
            for col in range(9):
                if board[row][col] != '.':
                    number = int(board[row][col])
                    addNumber(row, col, number)
#         final_config = None
        def backtrack(row=0, col=0):
            if row == 9:
#                 nonlocal final_config
#                 final_config = [row[:] for row in board]
                return True
            nextrow, nextcol = getNextCell(row, col)
            if board[row][col] != '.':
                return backtrack(nextrow, nextcol)
            if board[row][col] == '.':
                ret = False
                for number in range(1, 10):
                    if checkNumber(row, col, number):
                        addNumber(row, col, number)
                        if backtrack(nextrow, nextcol):
                            ret = True
                            break
                        removeNumber(row, col, number)
                return ret
        backtrack()
#         return final_config

In [170]:
board = [["5","3",".",".","7",".",".",".","."],["6",".",".","1","9","5",".",".","."],[".","9","8",".",".",".",".","6","."],["8",".",".",".","6",".",".",".","3"],["4",".",".","8",".","3",".",".","1"],["7",".",".",".","2",".",".",".","6"],[".","6",".",".",".",".","2","8","."],[".",".",".","4","1","9",".",".","5"],[".",".",".",".","8",".",".","7","9"]]

In [171]:
solver_37 = Solution37()
solver_37.solveSudoku(board)

[['5', '3', '4', '6', '7', '8', '9', '1', '2'],
 ['6', '7', '2', '1', '9', '5', '3', '4', '8'],
 ['1', '9', '8', '3', '4', '2', '5', '6', '7'],
 ['8', '5', '9', '7', '6', '1', '4', '2', '3'],
 ['4', '2', '6', '8', '5', '3', '7', '9', '1'],
 ['7', '1', '3', '9', '2', '4', '8', '5', '6'],
 ['9', '6', '1', '5', '3', '7', '2', '8', '4'],
 ['2', '8', '7', '4', '1', '9', '6', '3', '5'],
 ['3', '4', '5', '2', '8', '6', '1', '7', '9']]

In [172]:
board

[['5', '3', '4', '6', '7', '8', '9', '1', '2'],
 ['6', '7', '2', '1', '9', '5', '3', '4', '8'],
 ['1', '9', '8', '3', '4', '2', '5', '6', '7'],
 ['8', '5', '9', '7', '6', '1', '4', '2', '3'],
 ['4', '2', '6', '8', '5', '3', '7', '9', '1'],
 ['7', '1', '3', '9', '2', '4', '8', '5', '6'],
 ['9', '6', '1', '5', '3', '7', '2', '8', '4'],
 ['2', '8', '7', '4', '1', '9', '6', '3', '5'],
 ['3', '4', '5', '2', '8', '6', '1', '7', '9']]

## 1239. Maximum Length of a Concatenated String with Unique Characters

In [173]:
from collections import defaultdict

### self-written backtracking solution, passed all tests

In [192]:
class Solution1239:
    def maxLength(self, arr: List[str]) -> int:
        ret = 0
        chars = defaultdict(int)
        def backtrack(start=0):
            nonlocal ret
            count = sum(chars.values())
            ret = max(ret, count)
            for i in range(start, len(arr)):
                # placement
                valid = True
                for j, char in enumerate(arr[i]):
                    chars[char] += 1
                    if chars[char] > 1:
                        valid = False
                        for k in range(j, -1, -1):
                             chars[arr[i][k]] -= 1
                        break
                if valid:
                    backtrack(i + 1)
                    # removal
                    for char in arr[i]:
                        chars[char] -= 1
        backtrack()
        return ret

In [193]:
solver_1239 = Solution1239()
solver_1239.maxLength(["un","iq","ue"]), solver_1239.maxLength(arr = ["cha","r","act","ers"]),\
solver_1239.maxLength(["abcdefghijklmnopqrstuvwxyz"]), solver_1239.maxLength(["jnfbyktlrqumowxd","mvhgcpxnjzrdei"])

(4, 6, 26, 16)

## 126. Word Ladder II

### self-written BFS solution, time limit exceeded

In [201]:
import string
from collections import deque

In [215]:
class Solution126:
    def findLadders(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]:   
        ret = []
        words = set(wordList)
        if endWord not in wordList:
            return ret
        queue = deque([[beginWord]])
        found = False
        while queue:
            sequence = queue.popleft()
#             print(sequence)
            if not found or (found and len(sequence) < minLength):
                lastword = sequence[-1]
#                 print(lastword)
                wordsleft = words - set(sequence)
#                 print(wordsleft)
                valid_words = [lastword[:i] + char + lastword[i+1:] for i in range(len(lastword)) for char in string.ascii_lowercase]
#                 print(valid_words)
                for candidate_word in wordsleft:
                    if candidate_word in valid_words:
                        if candidate_word == endWord:
                            if not found:
                                found = True
                                minLength = len(sequence) + 1
                            ret.append(sequence + [candidate_word])
                        else:
                            queue.append(sequence + [candidate_word])
            else:
                break
        return ret 

In [217]:
solver_126 = Solution126()
solver_126.findLadders(beginWord = "hit", endWord = "cog", wordList = ["hot","dot","dog","lot","log","cog"]),\
solver_126.findLadders(beginWord = "hit", endWord = "cog", wordList = ["hot","dot","dog","lot","log"])

([['hit', 'hot', 'dot', 'dog', 'cog'], ['hit', 'hot', 'lot', 'log', 'cog']],
 [])

### one-way BFS solution in discussion area, passed all tests

In [233]:
class Solution126:
    def findLadders(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]:  
        tree, words, n = defaultdict(set), set(wordList), len(beginWord)
        if endWord not in wordList: return []
        found, currLayer, nextLayer = False, {beginWord}, set()
        while currLayer and not found:
#             print(currLayer)
            words -= set(currLayer)
            for x in currLayer:
                for y in [x[:i]+c+x[i+1:] for i in range(n) for c in string.ascii_lowercase]:
                    if y in words:
                        if y == endWord: 
                            found = True
                        else: 
                            nextLayer.add(y)
                        tree[x].add(y)
            currLayer, nextLayer = nextLayer, set()
        def constructPath(x): 
            # if a path ends at nowhere (i.e., != endWord, defaultdict will return empty tree[x])
            return [[x]] if x == endWord else [[x] + rest for y in tree[x] for rest in constructPath(y)]
#         print(tree)
        return constructPath(beginWord)

In [234]:
solver_126 = Solution126()
solver_126.findLadders(beginWord = "hit", endWord = "cog", wordList = ["hot","dot","dog","lot","log","cog"]),\
solver_126.findLadders(beginWord = "hit", endWord = "cog", wordList = ["hot","dot","dog","lot","log"])

([['hit', 'hot', 'dot', 'dog', 'cog'], ['hit', 'hot', 'lot', 'log', 'cog']],
 [])

### optimized two-way BFS solution in discussion area, passed all tests, genius solution

In [243]:
class Solution126:
    def findLadders(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]: 
        tree, words, n = defaultdict(set), set(wordList), len(beginWord)
        if endWord not in wordList: return []
        found, bq, eq, nq, rev = False, {beginWord}, {endWord}, set(), False
        while bq and not found:
#             print(bq, eq, nq)
            words -= set(bq)
            for x in bq:
                for y in [x[:i]+c+x[i+1:] for i in range(n) for c in string.ascii_lowercase]:
                    if y in words:
                        if y in eq: 
                            found = True
                        else: 
                            nq.add(y)
                        tree[y].add(x) if rev else tree[x].add(y) # keep record of flipping of bq and eq
            bq, nq = nq, set()
            if len(bq) > len(eq): 
                bq, eq, rev = eq, bq, not rev
        def bt(x): 
            return [[x]] if x == endWord else [[x] + rest for y in tree[x] for rest in bt(y)]
        return bt(beginWord)

In [244]:
solver_126 = Solution126()
solver_126.findLadders(beginWord = "hit", endWord = "cog", wordList = ["hot","dot","dog","lot","log","cog"]),\
solver_126.findLadders(beginWord = "hit", endWord = "cog", wordList = ["hot","dot","dog","lot","log"])

([['hit', 'hot', 'dot', 'dog', 'cog'], ['hit', 'hot', 'lot', 'log', 'cog']],
 [])

## 131. Palindrome Partitioning

### self-written backtracking solution after checking answers

In [247]:
class Solution131:
    def partition(self, s: str) -> List[List[str]]:
        def checkPalindrome(i, j):
            if i >= j:
                return True
            return s[i] == s[j] and checkPalindrome(i+1, j-1)
        ret = []
        choices = []
        def backtrack(start=0):
            if start == len(s):
                ret.append(choices[:])
            for i in range(start, len(s)):
                if checkPalindrome(start, i):
                    choices.append(s[start:i+1])
                    backtrack(i+1)
                    choices.pop()
        backtrack()
        return ret

In [249]:
solver_131 = Solution131()
solver_131.partition("aab"), solver_131.partition("abccba")

([['a', 'a', 'b'], ['aa', 'b']],
 [['a', 'b', 'c', 'c', 'b', 'a'],
  ['a', 'b', 'cc', 'b', 'a'],
  ['a', 'bccb', 'a'],
  ['abccba']])

## 93. Restore IP Addresses

### self-written backtracking solution

In [267]:
class Solution93:
    def restoreIpAddresses(self, s: str) -> List[str]:
        ret = []
        choices = []
        def backtrack(start=0):
            if len(choices) == 4:
#                 print(choices, start)
                if start == len(s):
                    ret.append(choices[:])
                else:
                    return
            # single digit
            if start < len(s):
                number = s[start]
                choices.append(number)
                backtrack(start + 1)
                choices.pop()
            if start + 2 <= len(s) and s[start] != '0':
                number = s[start:start+2]
                choices.append(number)
                backtrack(start + 2)
                choices.pop()
            if start + 3 <= len(s) and s[start] != '0' and int(s[start:start+3]) <= 255:
                number = s[start:start+3]
                choices.append(number)
                backtrack(start + 3)
                choices.pop()
        backtrack()
        return [".".join(choice) for choice in ret]

In [268]:
solver_93 = Solution93()
solver_93.restoreIpAddresses("25525511135")

['255.255.11.135', '255.255.111.35']

## 980. Unique Paths III

### self-written backtracking solution

In [288]:
class Solution980:
    def uniquePathsIII(self, grid: List[List[int]]) -> int:
        nrow = len(grid)
        ncol = len(grid[0])
        # search for start, end and non-obstacles
        must = set() # this must acts like constraints
        for row in range(nrow):
            for col in range(ncol):
                if grid[row][col] == 1:
                    start = (row, col)
                    must.add((row, col))
                elif grid[row][col] == 2:
                    end = (row, col)
                elif grid[row][col] == 0:
                    must.add((row, col))
        count = 0
#         print(start, end, must)
        def backtrack(row, col):
#             print(row, col)
            if row == -1 or row == nrow or col == -1 or col == ncol:
#                 print("out of boundary")
                return
            # end condition or reach stop too early
            if (row, col) == end:
                if not must:
                    nonlocal count
                    count += 1
#                     print("solution found")
                    return
#                 print("reach stop too early")
                return
            # either go back to start or run into obstacles or walk over empty cells twice
            if (row, col) not in must:
#                 print("go back to start or run into obstacles or walk over empty cells twice")
                return
#             print("start exploring")
            for nextrow, nextcol in [(row+1, col), (row-1, col), (row, col+1), (row, col-1)]:
                must.remove((row, col))
                backtrack(nextrow, nextcol)
                must.add((row, col))
        backtrack(*start)
        return count

In [291]:
solver_980 = Solution980()
solver_980.uniquePathsIII([[1,0,0,0],[0,0,0,0],[0,0,2,-1]]),\
solver_980.uniquePathsIII([[1,0,0,0],[0,0,0,0],[0,0,0,2]]),\
solver_980.uniquePathsIII([[0,1],[2,0]])

(2, 4, 0)

## 306. Additive Number

### self-written backtracking solution, passed all tests, difficult code debugging

In [360]:
class Solution306:
    def isAdditiveNumber(self, num: str) -> bool:
        """
        the bug was if num[start] != 0: instead of if num[start] != '0':
        OMG...
        """
        found = False
        def backtrack(prev2, prev1, start):
#             print(prev2, prev1, start)
            if start == len(num):
                if prev2 is not None:
#                     print(prev2, prev1, start)
                    nonlocal found
                    found = True
                    return
                return
            elif prev2 is None:
                # haven't collected enough numbers yet
                # need to iterate over all possible numbers
                number = int(num[start])
                # prevent edge case like "10"
                # if two numbers are found and reached the end this is considered invalid
                if prev1 is not None and start+1 == len(num):
                    return
#                 print(prev1, number, start+1)
                backtrack(prev1, number, start+1)
                if num[start] != '0':
#                     print('entering iteration')
                    for i in range(start+1, len(num)):
                        if not found:
                            number = int(num[start:i+1])
                            # if two numbers are found and reached the end this is considered invalid
                            if prev1 is not None and i+1 == len(num):
                                return
#                             print(prev1, number, i+1)
                            backtrack(prev1, number, i+1)
            else:
                # faster searching
                target = prev1 + prev2
                i = len(str(target))
                if start + i <= len(num):
                    number = int(num[start:start+i])
#                     print(prev2, prev1, num[start:start+i])
                    if number == target and not found:
#                         print(prev2, prev1, num[start:start+i])
                        backtrack(prev1, number, start+i)
        backtrack(None, None, 0)
        return found

In [361]:
solver_306 = Solution306()
# solver_306.isAdditiveNumber("112358"), solver_306.isAdditiveNumber("199100199"),\
solver_306.isAdditiveNumber("10"), solver_306.isAdditiveNumber("1023")

(False, False)

## 89. Gray Code

### self-written backtracking solution, passed all tests but very slow

In [382]:
class Solution89:
    def grayCode(self, n: int) -> List[int]:
        allNumbers = set()
        base = ['0'] * n
        def backtrackBuildSet(start=0):
            allNumbers.add("".join(base))
            for i in range(start, n):
                base[i] = '1'
                backtrackBuildSet(i+1)
                base[i] = '0'
        backtrackBuildSet()
#         print(allNumbers)
        choices = ['0'*n]
        allNumbers.remove('0'*n)
        found = False
        def toggleDigit(number: List[str], i):
            if number[i] == '1':
                number[i] = '0'
            else:
                number[i] = '1'
            return number
        def backtrack():
            if not allNumbers:
                nonlocal found
                found = True
                return True
            lastNumber = choices[-1]
            ret = False
            for i in range(n):
                newNumber = toggleDigit(list(lastNumber), i)
                newNumber = "".join(newNumber)
                if newNumber in allNumbers and not found:
                    allNumbers.remove(newNumber)
                    choices.append(newNumber)
                    if backtrack():
                        ret = True
                        break
                    choices.pop()
                    allNumbers.add(newNumber)
            return ret
        backtrack()
        return [int(number, 2) for number in choices]

In [383]:
solver_89 = Solution89()
solver_89.grayCode(4)

[0, 8, 12, 4, 6, 14, 10, 2, 3, 11, 15, 7, 5, 13, 9, 1]